# Testcase with PyTorch

In [49]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

## Load preprocessed data

In [151]:
data_pd = pd.read_csv('./data/housing_.csv')

data = data_pd.values
columns = data_pd.columns
data = torch.tensor(data, dtype=torch.float32)

train_data, test_data = train_test_split(data,train_size=0.8,test_size=0.2)
print(train_data.shape, test_data.shape)

train_data_df = pd.DataFrame(train_data)
test_data_df = pd.DataFrame(test_data)
train_data_df.columns = columns
test_data_df.columns = columns
print(train_data_df.head())
train_data_df.to_csv('./data/train_data.csv',index=True)
test_data_df.to_csv('./data/test_data.csv', index=True)

torch.Size([404, 14]) torch.Size([102, 14])
   CrimeRate  ResidentialLandZone  IndustryProportion  RiverTract  \
0    5.58107                  0.0               18.10         0.0   
1    9.92485                  0.0               18.10         0.0   
2    0.06860                  0.0                2.89         0.0   
3    6.53876                  0.0               18.10         1.0   
4    0.09512                  0.0               12.83         0.0   

   AirPollution  AvgRooms  OlderBuildingsPercent  DistanceToEmployment  \
0         0.713     6.436              87.900002                2.3158   
1         0.740     6.251              96.599998                2.1980   
2         0.445     7.416              62.500000                3.4952   
3         0.631     7.016              97.500000                1.2024   
4         0.437     6.286              45.000000                4.5026   

   HighwayAccess  PropertyTaxRate  PupilTeacherRatio  BlackPopulation  \
0           24.0       

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, fp):
        data = pd.read_csv(fp)
        self.data = data.drop('HousePrice',axis=1).astype(float)
        
        self.target = data['HousePrice'].astype(float)
    

    
       

    def __len__(self):
        return self.data.shape[0]
    
    def __shape__(self):
        return self.data.shape

    def __getitem__(self, idx):
        data_x = self.data.iloc[idx] 
        data_y = self.target.iloc[idx]
        return torch.tensor(data_x,dtype=torch.float32),torch.tensor(data_y,dtype=torch.float32)

In [143]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train(model, optimizer, loss_fn, data_loader, n_epochs=100):

    for epoch in range(n_epochs):
        model.train()
        loss_ = 0
        for X, y in data_loader:
        
            output = model(X)
            loss = loss_fn(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_+=loss

        if (epoch%100)==0:
            print(f'Epoch: {epoch} | Loss: {loss.item():.4f}')






In [153]:
train_dataset = CustomDataset(fp='./data/train_data.csv')
test_dataset = CustomDataset(fp='./data/test_data.csv')


In [147]:

train_dataloader = DataLoader(train_dataset, batch_size=16,shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

model = MLP(18,8,1)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000001)

train(model, optimizer, loss_fn, train_dataloader,n_epochs=1000)

/var/folders/ms/1fs2z5sj60l86m3vc7bkzmc00000gn/T/ipykernel_58660/3175185563.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(data_x,dtype=torch.float32),torch.tensor(data_y,dtype=torch.float32)


Epoch: 0 | Loss: 174257.7969
Epoch: 100 | Loss: 163344.5000
Epoch: 200 | Loss: 192306.5781
Epoch: 300 | Loss: 180987.8281
Epoch: 400 | Loss: 188886.4219
Epoch: 500 | Loss: 186303.7812
Epoch: 600 | Loss: 214704.7500
Epoch: 700 | Loss: 172635.6406
Epoch: 800 | Loss: 198253.6562
Epoch: 900 | Loss: 193630.8594
